## Import and Configure Everything We Need

In [1]:
%matplotlib inline

from collections import defaultdict as ddict, OrderedDict as odict
from typing import Any, Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from rdkit.Chem import PandasTools, AllChem as Chem, Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import deepchem as dc
import torch

from rdkit import RDLogger

RDLogger.DisableLog('rdApp.*') 

import basic as b
import chemprop_ish as c

pd.set_option('display.float_format', lambda x: '%.3f' % x)  # Display floats without scientific notation

# In many cases NaN
not_used_desc = ['MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge']

# Create a descriptor calculator for all RDKit descriptors except the ones above
desc_calc = MolecularDescriptorCalculator([x for x in [x[0] for x in Descriptors.descList] if x not in not_used_desc])

RDKit WARNING: [19:18:12] Enabling RDKit 2019.09.3 jupyter extensions


SyntaxError: invalid syntax (basic.py, line 138)

---
## Loading Precombined Dataset

In [ ]:
data = pd.read_csv('combisolv_exp2.csv')
solute = data['smiles_solute'].tolist()
solvent = data['smiles_solvent'].tolist()
pka = data['dGsolv_avg [kcal/mol]'].tolist()
sol_solv = [[x,y] for x,y in zip(solute,solvent)]
#preprocess pka too

In [ ]:
H_list = []
for x in range(len(sol_solv)):
    if sol_solv[x][0] in ["[H][H]","[2H][2H]","[HH]"]:
        H_list.append(x)
for x in sorted(H_list, reverse = True):
    del sol_solv[x]
    del pka[x]

In [70]:
sol_solv_mol = [[Chem.MolFromSmiles(x) for x in y] for y in sol_solv]

In [71]:
len(sol_solv_mol)

8700

In [72]:
#check for mols with 0 heavy atoms
for x in sol_solv_mol:
    for y in x:
        if y.GetNumHeavyAtoms() == 0:
            print(Chem.MolToSmiles(y))

## Calculating Descriptors and Fingerprints
- 196/200 RDKit descriptors
- Morgan FP with radius=3 and useFeatures=True (FMorgan3)

In [87]:
descs, fmorgan3, descs_fmorgan3 = b.calc_xy_data(sol_solv_mol)

In [88]:
scaler = StandardScaler()
descs = scaler.fit_transform(descs)
fmorgan3 = scaler.fit_transform(fmorgan3)
descs_fmorgan3 = scaler.fit_transform(descs_fmorgan3)
#pka = scaler.fit_transform(pka)

In [97]:
print(len(descs[0]))

392


---
## Training Random Forest, Support Vector Machine (two configurations) and Multilayer Perceptron (three configurations)
#### Using the following training sets with 5-fold cross-validation (shuffled)
1. RDKit descriptor set
2. FMorgan3
3. RDKit descriptor set + FMorgan3

### Prepare for Training

In [89]:
seed = 24

est_jobs = 12
verbose = False

y_data = np.array(pka)
desc_sets = [[descs, 'Desc'],[fmorgan3, 'FMorgan3'],[descs_fmorgan3, 'Desc_FMorgan3']]

models = ddict(odict)  # estimator => training set => [model, scaler]

def train_all_sets(est_cls, params, name, torch_model):
    for x_data, set_name in desc_sets:
        models[name][set_name] = b.train_cv_model(est_cls, x_data, y_data, params, seed, torch_model=torch_model)

In [90]:
def generate_score_board(name):
    print(f'{name} CV Scores:')
    for ts, m in models[name].items():
        print(f'\t{ts}')
        for k, v in m.cv_scores.items():
            print(f'\t\t- {k}: {np.mean(v):.3f} ± {np.std(v):.3f}')

---
### RandomForest (n_estimators=1000)

In [91]:
est_cls = RandomForestRegressor
rf_params = dict(n_estimators=1000, n_jobs=est_jobs, verbose=verbose, random_state=seed)
name = 'RandomForest (n_estimators=1000)'

train_all_sets(est_cls, rf_params, name, False)
generate_score_board(name)

RandomForest (n_estimators=1000) CV Scores:
	Desc
		- mean_absolute_error: 0.313 ± 0.008
		- rmse: 0.671 ± 0.034
		- r2_score: 0.978 ± 0.002
	FMorgan3
		- mean_absolute_error: 0.717 ± 0.022
		- rmse: 1.380 ± 0.049
		- r2_score: 0.907 ± 0.006
	Desc_FMorgan3
		- mean_absolute_error: 0.313 ± 0.008
		- rmse: 0.672 ± 0.036
		- r2_score: 0.978 ± 0.002


---
### SupportVectorMachine (gamma='scale')

In [92]:
est_cls = SVR
svr_params = dict(cache_size=4096, verbose=verbose)
name = 'SupportVectorMachine (gamma="scale")'

train_all_sets(est_cls, svr_params, name, False)
generate_score_board(name)

SupportVectorMachine (gamma="scale") CV Scores:
	Desc
		- mean_absolute_error: 0.540 ± 0.015
		- rmse: 1.482 ± 0.095
		- r2_score: 0.892 ± 0.013
	FMorgan3
		- mean_absolute_error: 1.330 ± 0.027
		- rmse: 2.510 ± 0.049
		- r2_score: 0.692 ± 0.007
	Desc_FMorgan3
		- mean_absolute_error: 1.017 ± 0.020
		- rmse: 2.212 ± 0.074
		- r2_score: 0.761 ± 0.012


---
### Multi Layer Perceptron (early_stopping=False, hidden_layer_sizes=(500, 500))

In [93]:
est_cls = MLPRegressor
mlp_params = dict(hidden_layer_sizes=(500, 500), verbose=verbose, random_state=seed)
name = 'Multi Layer Perceptron (early_stopping=False, hidden_layer_sizes=(500, 500))'

train_all_sets(est_cls, mlp_params, name, False)
generate_score_board(name)

Multi Layer Perceptron (early_stopping=False, hidden_layer_sizes=(500, 500)) CV Scores:
	Desc
		- mean_absolute_error: 0.290 ± 0.026
		- rmse: 0.529 ± 0.035
		- r2_score: 0.986 ± 0.002
	FMorgan3
		- mean_absolute_error: 0.858 ± 0.039
		- rmse: 1.693 ± 0.101
		- r2_score: 0.860 ± 0.015
	Desc_FMorgan3
		- mean_absolute_error: 0.434 ± 0.024
		- rmse: 1.004 ± 0.112
		- r2_score: 0.950 ± 0.011


---
### XGradientBoost

In [94]:
est_cls = xgb.XGBRegressor
xgb_params = dict(verbosity=2 if verbose else 0, random_state=seed, n_jobs=est_jobs)
name = 'XGradientBoost'

train_all_sets(est_cls, xgb_params, name, False)
generate_score_board(name)

XGradientBoost CV Scores:
	Desc
		- mean_absolute_error: 0.299 ± 0.007
		- rmse: 0.606 ± 0.040
		- r2_score: 0.982 ± 0.002
	FMorgan3
		- mean_absolute_error: 0.870 ± 0.027
		- rmse: 1.429 ± 0.043
		- r2_score: 0.900 ± 0.006
	Desc_FMorgan3
		- mean_absolute_error: 0.334 ± 0.006
		- rmse: 0.641 ± 0.040
		- r2_score: 0.980 ± 0.002


---
## Training torch models
#### Using the following training sets with 5-fold cross-validation (shuffled)
1. Sol / solvent pairs

---
### Torch models

In [7]:
seed = 24
verbose = False

y_data = torch.Tensor(pka)
x_data = sol_solv

In [8]:
args = c.TrainArgs()
args.__dict__.update({"depth":3, "dropout":0.2})
est_cls = c.double_MPNN(args)
name = 'MPNN'

b.train_cv_model(est_cls, x_data, y_data, random_state=seed)
generate_score_board(name)

TypeError: train_cv_model() missing 1 required positional argument: 'random_state'